## Starting off with Pyspark with reading and writiing different file types, checking schema, changing schema

### Import Pyspark and create SparkSession.

This is the first thing to do when working with pyspark. The spark variable will also provide access to a UI to monitor jobs.

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadwriteVal").getOrCreate()

Check out the number of cores we are working with

In [2]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
cores

1

#### Reading CSV 

In [3]:
path = "Datasets/"

students = spark.read.csv(path+"students.csv",inferSchema=True,header=True)

In [4]:
students

DataFrame[gender: string, race/ethnicity: string, parental level of education: string, lunch: string, test preparation course: string, math score: int, reading score: int, writing score: int]

In [5]:
students.show(5)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

Not the best way to view what we have in the spark dataframe. To have a much cleaner view to the content we cand do something.

In [6]:
students.limit(5).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Reading parquet files

There's different ways to do so depending on what we want to read, how much we want to read

Reading a single parquet file

In [7]:
parquet = spark.read.parquet(path+"users1.parquet")

In [8]:
parquet.limit(4).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 01:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 11:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-02 19:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-02 18:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,


Reading all parquet files in the path with name starting with 'users'

In [9]:
partitioned = spark.read.parquet(path + "users*")

In [10]:
parquet.limit(4).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 01:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 11:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-02 19:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-02 18:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,


Reading selective parquet files

In [11]:
users1_2 = spark.read.parquet(path+"users1.parquet",path+"users2.parquet")
users1_2.limit(4).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 01:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 11:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-02 19:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-02 18:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,


#### Viewing schema of a dataframe

In [12]:
students.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



#### Viewing columns of a dataframe

In [13]:
students.columns

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

Get similar information to what you get from printSchema() shown above with describe()

In [14]:
students.describe()

DataFrame[summary: string, gender: string, race/ethnicity: string, parental level of education: string, lunch: string, test preparation course: string, math score: string, reading score: string, writing score: string]

#### We can also check the data type of a particular column from the dataframe schema as shown

In [15]:
students.schema['math score'].dataType

IntegerType

#### Select a subset of columns using select

In [16]:
students.select("math score","reading score").show(5)

+----------+-------------+
|math score|reading score|
+----------+-------------+
|        72|           72|
|        69|           90|
|        90|           95|
|        47|           57|
|        76|           78|
+----------+-------------+
only showing top 5 rows



#### Get some statistics from the columns in the dataframe

In [17]:
students.select("math score","reading score").summary("count","min","max").show()

+-------+----------+-------------+
|summary|math score|reading score|
+-------+----------+-------------+
|  count|      1000|         1000|
|    min|         0|           17|
|    max|       100|          100|
+-------+----------+-------------+



### Importing from sql types as now we want to play with the data types of the columns and change schema

In [18]:
from pyspark.sql.types import *

#### Describing the new schema for the dataframe

In [19]:
data_schema = [StructField("name",StringType(),True),
              StructField("email",StringType(),True),
              StructField("city",StringType(),True),
              StructField("mac",StringType(),True),
              StructField("timestamp",DateType(),True),
              StructField("creditcard",StringType(),True)]

#### Creating a structure object out of the defined schema

In [20]:
final_struc = StructType(fields = data_schema)

#### Reading csv with the defined schema instead of letting pyspark infer schema on read

In [22]:
people = spark.read.json(path+"people.json",schema=final_struc)

In [23]:
people.limit(4).toPandas()

,name,email,city,mac,timestamp,creditcard
0,None,None,None,None,None,None
1,Keeley Bosco,katlyn@jenkinsmaggio.net,Lake Gladysberg,08:fd:0b:cd:77:f7,2015-04-25,1228-1221-1221-1431
2,Rubye Jerde,juvenal@johnston.name,None,90:4d:fa:42:63:a2,2015-04-25,1228-1221-1221-1431
3,Miss Darian Breitenberg,None,None,f9:0e:d3:40:cb:e9,2015-04-25,None


In [24]:
people.printSchema()

root
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- city: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- timestamp: date (nullable = true)
 |-- creditcard: string (nullable = true)



### Writing Dataframes

In [25]:
students.write.mode("overwrite").csv(path+"write_test.csv")

In [26]:
users1_2.write.mode("overwrite").parquet(path+"parquet")

#### Writing a parquet file that is partitioned on gender column. The output is a folder that has multiple parquet files, one for each gender value in the column. 

If we don't use option("basepath",path), the resulting parquet files won't contain the column gender in t

In [42]:
users1_2.write.option("basepath",path).mode("overwrite").partitionBy("gender").parquet(path+"parquet_part_gender")

In [43]:
values = [("Pear",10),("Orange",13),("Peach",5)]
df = spark.createDataFrame(values,["fruit","quantity"])
df.show()

+------+--------+
| fruit|quantity|
+------+--------+
|  Pear|      10|
|Orange|      13|
| Peach|       5|
+------+--------+

